## Takens h2t diffusion map

### setup

In [ ]:
# Read in required
import os
import matplotlib as mpl
if os.environ.get('DISPLAY','') == '':
    print('no display found. Using non-interactive Agg backend')
    mpl.use('Agg')
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

import sys
import math
import time
import pickle
from copy import deepcopy

import numpy as np

import sklearn.preprocessing as pre
from sklearn.model_selection import train_test_split
from sklearn import manifold
from sklearn.metrics.pairwise import euclidean_distances

from scipy.linalg import orthogonal_procrustes
from scipy.linalg import eigh

from collections import OrderedDict

import mdtraj as md
import nglview as nv

In [ ]:
showPlots=1
useMagics=1
if useMagics:
    %matplotlib inline
    #%matplotlib notebook
    %load_ext autoreload
    %autoreload 2'''

In [ ]:
# fix random seed for reproducibility
randstate = 200184
np.random.seed(randstate)

### loading simulation trajectory

In [ ]:
trj_dir = "../C24/"
pdb_file = os.path.join(trj_dir, "after_pr_protein.pdb")
trj_file = os.path.join(trj_dir, "md_protein_fit.xtc")

In [ ]:
# loading
traj = md.load(trj_file, top=pdb_file)

In [ ]:
# rot+trans alignment of trajectory to first frame
traj.center_coordinates(mass_weighted=False)
traj.superpose(traj[0])

In [ ]:
print('How many frames? %s' % traj.n_frames)
print('How many atoms?    %s' % traj.n_atoms)
print(traj.xyz.shape)

In [ ]:
# visualizing
view = nv.show_mdtraj(traj)
view.remove_cartoon()
view.add_ball_and_stick()
view

### loading principal moments of gyration tensor

In [ ]:
npzfile = np.load('../0_traj2pdh2t/princMom.npz')
sorted(npzfile.files)
princMom = npzfile['princMom']
Rg = npzfile['Rg']

print(princMom.shape)
print(Rg.shape)

### loading h2t timeseries

In [ ]:
npzfile = np.load('../0_traj2pdh2t/h2t.npz')
sorted(npzfile.files)
h2t = npzfile['h2t']

In [ ]:
# flattening
h2t = np.reshape(h2t,-1)

In [ ]:
# no. of frames
nFrames = len(h2t)
print('nFrames = %d' % nFrames)

### estimation of delay time using (i) autocorrelation and (ii) MI 

Ref: A. M. Fraser and H. L. Swinney, Independent coordinates for strange attractors from mutual information, Phys. Rev. A 33, 1134 (1986)

#### (i) autocorrelation

In [ ]:
tau_array = np.arange(0,5000,1)

h2t_mu = np.mean(h2t)
h2t_sigma = np.std(h2t)

h2t_mz = h2t-h2t_mu

AC_array = []

for tau in tau_array:
    AC_array.append( np.mean( np.multiply(h2t_mz[0:len(h2t_mz)-tau], h2t_mz[tau:len(h2t_mz)]) ) / (h2t_sigma * h2t_sigma) )

fig, ax = plt.subplots()
ax.plot(tau_array, AC_array, 'b-', marker='o')
ax.plot([tau_array[0],tau_array[-1]], [0,0], 'k:')
ax.set_xlabel("delay time, tau (steps)")
ax.set_ylabel("autocorrelation, R(tau)")
ax.figure.savefig("AC.pdf")
plt.draw()

#### (ii) mutual information

In [ ]:
tau_array = np.arange(0,1000,10)

bins=2500
MI_array = []

def shan_entropy(c):
    c_normalized = c / float(np.sum(c))
    c_normalized = c_normalized[np.nonzero(c_normalized)]
    H = -sum(c_normalized* np.log2(c_normalized))  
    return H

for tau in tau_array:
    
    x = h2t[0:len(h2t)-tau]
    y = h2t[tau:len(h2t)]
    
    c_xy = np.histogram2d(x,y,bins)[0]
    c_x = np.histogram(x,bins)[0]
    c_y = np.histogram(y,bins)[0]
    
    H_x = shan_entropy(c_x)
    H_y = shan_entropy(c_y)
    H_xy = shan_entropy(c_xy)
    
    MI = H_x + H_y - H_xy
    
    MI_array.append(MI)

In [ ]:
fig, ax = plt.subplots()
ax.plot(tau_array, MI_array, 'r-', marker='o')
ax.set_xlabel("delay time, tau (steps)")
ax.set_ylabel("mutual information, MI(tau)")
ax.set_ylim(3,4)
ax.figure.savefig("MI.pdf")
plt.draw()

In [ ]:
# specifying delay time tau (in steps)
tau = 150

### estimation of delay dimensionality using false nearest neighbors

Ref: L. Cao, Practical method for determining the minimum embedding dimension of a scalar time series, Phys. D (Amsterdam, Neth.) 110, 43 (1997)

In [ ]:
# performing E1d analysis using specified value of tau (steps)
Ed=[]

dMax = 25
for d in range(1,dMax):
    
    # building d-dim delay embedding of len(h2t)-(d-1)*tau delay points with delay time of tau (in steps)
    print 'd=%d' % d
    delayVecs = np.reshape(h2t[0:len(h2t)-(d-1)*tau],(len(h2t)-(d-1)*tau,1))
    for ii in range(1,d):
        delayVecs = np.concatenate((delayVecs,np.reshape(h2t[ii*tau:len(h2t)-(d-1)*tau+ii*tau],(len(h2t)-(d-1)*tau,1))), axis=1)
    print delayVecs
    print delayVecs.shape
    
    # subsampling
    startIdx_SS=0
    stopIdx_SS=100000#len(h2t)-(dMax+1)
    stride_SS=10
    if startIdx_SS<0 | stopIdx_SS>delayVecs.shape[0]:
        print 'ERROR: subsampling illegal points'
        break
    delayVecs_ss = delayVecs[startIdx_SS:stopIdx_SS:stride_SS,:]
    print delayVecs_ss.shape
    if delayVecs_ss.shape[0]>10000:
        print 'ERROR: pairwise distance matrix exceeds 10k-by-10k, RAM maxout likely'
        break
    
    # computing all pairwise distances between delay vectors
    P = euclidean_distances(delayVecs_ss,delayVecs_ss)
    idx = np.arange(0,delayVecs_ss.shape[0],1)
    
    # computing a_id
    # N.B. Using idxNN_d and deltaNN_d from last iteration prior to update
    if d >= 2:
        Ed.append( np.mean( np.divide(P[idx,idxNN_d], deltaNN_d) ) )
    
    # identifying non-zero nearest neighbor of each point
    P[np.where(P==0)]=np.inf
    idxNN_d = np.argmin(P, axis=1)
    deltaNN_d = P[idx,idxNN_d]

Ed = np.array(Ed)
print Ed
E1d = np.divide( Ed[1:len(Ed)], Ed[0:len(Ed)-1] )
print E1d

fig, ax = plt.subplots()
ax.plot(np.arange(1,len(E1d)+1), E1d, 'k-', marker='o')
ax.set_xlabel("delay dim")
ax.set_ylabel("E1d")
ax.figure.savefig("E1d.pdf")
plt.draw()

In [ ]:
# specifying delay dimensionality
dim = 5 

### constructing delay vectors

In [ ]:
# specifying delay time tau (in steps) and delay dimensionality dim
tau = 150
dim = 5

nObs = len(h2t)
print('nObs = %d' % nObs)

In [ ]:
# constructing delay vectors
delayVecs = np.reshape(h2t[0:len(h2t)-(dim-1)*tau],(len(h2t)-(dim-1)*tau,1))
for ii in range(1,dim):
    delayVecs = np.concatenate((delayVecs,np.reshape(h2t[ii*tau:len(h2t)-(dim-1)*tau+ii*tau],(len(h2t)-(dim-1)*tau,1))), axis=1)
# converting to backward delay embeddings [v(t), v(t-tau), v(t-2*tau), ... , v(t-(dim-2)*tau), v(t-(dim-1)*tau)]
print(delayVecs)

nVecs = delayVecs.shape[0]
print('nVecs = %d' % nVecs)
print('delay dim = %d' % dim)

In [ ]:
# plotting delay vectors

plot_stride = 100

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
i=0
j=2
k=4
ax.scatter(delayVecs[::plot_stride,i], delayVecs[::plot_stride,j], delayVecs[::plot_stride,k])
ax.set_xlabel('delayVec_' + str(i))
ax.set_ylabel('delayVec_' + str(j))
ax.set_zlabel('delayVec_' + str(k))
if showPlots:
    plt.draw()
    plt.show()
plt.close()

### subsampling trajectory to manageable size for dMaps

In [ ]:
startFrame = 0
stopFrame = 100000
skipper = 10
delayVecs_ss = deepcopy(delayVecs[startFrame:stopFrame:skipper])
princMom_ss = princMom[startFrame:stopFrame:skipper]
Rg_ss = Rg[startFrame:stopFrame:skipper]
h2t_ss = h2t[startFrame:stopFrame:skipper]

### generating time inverted subsampled trajectories

In [ ]:
delayVecs_ss_flip = np.fliplr(delayVecs_ss)

### computing pairwise Euclidean distances between all frames of delayVecs_ss

symmetrized (minimized) under temporal flip

In [ ]:
N = delayVecs_ss.shape[0]

P = np.minimum( euclidean_distances(delayVecs_ss,delayVecs_ss), euclidean_distances(delayVecs_ss_flip,delayVecs_ss) )

outfile='P.npz'
np.savez(outfile, P=P)

### epsilon scan

In [ ]:
# scanning for appropriate gaussian kernal width
log_eps_array = np.arange(-6,6,1)
sumA = np.empty_like(log_eps_array)

for i in range(len(log_eps_array)):
    eps = np.exp(log_eps_array[i])
    sumA[i] = np.sum(np.exp(-P**2/(2*eps)),axis=None)

In [ ]:
# plotting
fig, ax = plt.subplots()
ax.scatter(log_eps_array, np.log(sumA))
ax.plot([log_eps_array[0],log_eps_array[-1]],[np.log(N**2),np.log(N**2)],'k:')
ax.plot([log_eps_array[0],log_eps_array[-1]],[np.log(N),np.log(N)],'k:')
ax.set_xlabel("log(eps)")
ax.set_ylabel("log(sumA)")
if showPlots:
    plt.draw()
    plt.show()

In [ ]:
# selecting
eps = np.exp(0)

### running diffusion maps

In [ ]:
# dMaps

outfile='P.npz'
npzfile = np.load(outfile)
sorted(npzfile.files)
P = npzfile['P']

P = np.exp(-P**2/(2*eps))
D = np.sum(P,axis=1)
P = np.matmul(np.matmul(np.diag(D**(-0.5)),P),np.diag(D**(-0.5))) # constructing Ms as symmetric matrix for diagonalization

#assert np.allclose(P, P.T)

nEvals=20
lamb, psi = eigh(P,eigvals=(P.shape[0]-nEvals,P.shape[0]-1)) # scipy eigh to specify only computation of leading evals
psi = np.matmul(np.diag(D**(-0.5)),psi) # converting evecs of Ms to evecs of M; M and Ms share evals

idx_sort = np.flip(np.argsort(lamb))

lamb = lamb[idx_sort]
psi = psi[:,idx_sort]

In [ ]:
# plotting eval spectrum
fig = plt.figure()
ax = fig.add_subplot(111)
ax.bar(np.arange(2,len(lamb)+1), lamb[1:])
ax.set_xlabel('eval idx')
ax.set_ylabel('eval')
plt.xticks(np.arange(2, len(lamb)+1, step=1))
if showPlots:
    plt.draw()
    plt.show()
fig.savefig('evals.png', dpi=300)
plt.close()

In [ ]:
# plotting embedding

plot_stride = 1

#colorMap = h2t_ss[::plot_stride].reshape(-1)
#colorMap = Rg_ss[::plot_stride].reshape(-1)
colorMap = princMom_ss[::plot_stride,2]

fig = plt.figure()
ax = fig.add_subplot(111)
i=1
j=2
ax.scatter(psi[::plot_stride,i], psi[::plot_stride,j], c=colorMap)
ax.set_xlabel('psi_' + str(i+1))
ax.set_ylabel('psi_' + str(j+1))
plt.xlim([np.min(psi[:,i])*1.1,np.max(psi[:,i])*1.1])
plt.ylim([np.min(psi[:,j])*1.1,np.max(psi[:,j])*1.1])
if showPlots:
    plt.draw()
    plt.show()
fig.savefig('evecs_23.png', dpi=300)
plt.close()

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
i=2
j=3
k=4
ax.scatter(psi[::plot_stride,i], psi[::plot_stride,j], psi[::plot_stride,k], c=colorMap)
ax.set_xlabel('psi_' + str(i+1))
ax.set_ylabel('psi_' + str(j+1))
ax.set_zlabel('psi_' + str(k+1))
if showPlots:
    plt.draw()
    plt.show()
fig.savefig('evecs_234.png', dpi=300)
plt.close()

### Nystrom extension

embedding all remaining points

In [ ]:
# Nystrom extension applied to all frames in data set

z = []

start = time.time()

print('Commencing Nystrom extension...')
for i in range(delayVecs.shape[0]):
    Euc_i = np.minimum( euclidean_distances(delayVecs_ss,delayVecs[i].reshape(1,-1)), euclidean_distances(delayVecs_ss_flip,delayVecs[i].reshape(1,-1)) )

    Euc_i = np.exp(-Euc_i**2/(2*eps))

    Euc_i /= np.sum(Euc_i)

    psi_Nystrom = np.divide( np.matmul(Euc_i.T,psi), lamb)
    
    z.append(psi_Nystrom)
    
    if np.mod(i+1,100) == 0:
        print('\tNystromed %d of %d frames...' % (i+1,traj.n_frames))

z = np.array(z)
z = z.reshape(z.shape[0],-1)

print('DONE!')
print('')

end = time.time()
print("Elapsed time %.2f (s)" % (end - start))

### saving data

In [ ]:
# saving dMap embedding of all data
np.savez('data.npz', z=z)
np.savez('nystrom.npz', delayVecs_ss=delayVecs_ss,delayVecs_ss_flip=delayVecs_ss_flip,z=z,psi=psi, lamb=lamb, eps=eps)

In [ ]:
# plotting embedding w/ Nystrom

plot_stride = 100

colorMap = princMom[:z.shape[0]:plot_stride,1]

fig = plt.figure()
ax = fig.add_subplot(111)
i=1
j=2
ax.scatter(z[::plot_stride,i], z[::plot_stride,j], c=colorMap)
ax.set_xlabel('psi_' + str(i+1))
ax.set_ylabel('psi_' + str(j+1))
plt.xlim([np.min(z[:,i])*1.1,np.max(z[:,i])*1.1])
plt.ylim([np.min(z[:,j])*1.1,np.max(z[:,j])*1.1])
if showPlots:
    plt.draw()
    plt.show()
fig.savefig('evecs_23_Nystrom.png', dpi=300)
plt.close()

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
i=1
j=2
k=3
ax.scatter(z[::plot_stride,i], z[::plot_stride,j], z[::plot_stride,k], c=colorMap)
ax.set_xlabel('psi_' + str(i+1))
ax.set_ylabel('psi_' + str(j+1))
ax.set_zlabel('psi_' + str(k+1))
if showPlots:
    plt.draw()
    plt.show()
fig.savefig('evecs_234_Nystrom.png', dpi=300)
plt.close()